In [ ]:
%matplotlib inline

# Tools for capturing filtering and analyzing data
import pandas as pd
import numpy as np

# Import widgets to add interactivity
from ipywidgets import interact_manual, interact, fixed, interactive_output
import ipywidgets as widgets
from IPython.display import display

# Interactive Report Generator

## Report parameters

Fill the following values so that reports can be generated:

In [ ]:
import os

list_of_reports = os.listdir('datasets')
list_of_reports = list(filter(lambda file: file.endswith('.csv'), list_of_reports))

In [ ]:
reportFileName = widgets.Combobox(
    value="ToProcess.csv",
    placeholder='Choose Report',
    options=list_of_reports,
    description='Report:',
    ensure_option=True,
    disabled=False
)
numberOfRowsToSkip = widgets.IntText(
    value=0,
    description='Number of rows to skip:',
    disabled=False
)
dateColumnName = widgets.Text(
    value='Time',
    description='Date column',
    disabled=False
)
nodeColumnName = widgets.Text(
    value='Huawei_eUtranCell',
    description='Node name column',
    disabled=False
)

In [ ]:
# Get report related scripts
import scripts.report_util as r_util

def preprocess_report_data(report, dateColumnName, nodeColumnName):
    report[dateColumnName] = r_util.convert_plain_to_datetime(report[dateColumnName], '%Y-%m-%d %H:%M:%S')
    report = r_util.get_max_per_day(report, nodeColumnName, dateColumnName)
    
    return report

def process_report(reportFileName, numberOfRowsToSkip, dateColumnName, nodeColumnName):
    report = r_util.read_report(reportFileName, numberOfRowsToSkip)
    report = preprocess_report_data(report, dateColumnName, nodeColumnName)
    report = r_util.get_pivot_table_per_kpi(report, dateColumnName, nodeColumnName)
    
    return report

def get_reports_per_kpi_per_node(pivot_table, nodeForReport, kpiForReport, surpassLimit, underpassLimit):
    current_report_kpi_node = pivot_table[kpiForReport].loc[pivot_table.index == nodeForReport]
    grouped_report = r_util.group_by_week(current_report_kpi_node)
    
    mean_week = r_util.get_mean_per_week_report(grouped_report)
    
    surpass_report = r_util.get_times_in_week_day_surpassed_last_week_mean_report(grouped_report, surpassLimit)
    underpass_report = r_util.get_times_in_week_day_below_last_week_mean_report(grouped_report, underpassLimit)
    
    r_util.get_plot_per_node_with_mean(current_report_kpi_node)
    
    
    return current_report_kpi_node, mean_week, surpass_report, underpass_report
    #return current_report_kpi_node

In [ ]:
w = interact(process_report, reportFileName=reportFileName, numberOfRowsToSkip=numberOfRowsToSkip, dateColumnName=dateColumnName, nodeColumnName=nodeColumnName).widget

In [ ]:
nodeForReport = widgets.Combobox(
    value=list(w.result.index)[0],
    options = list(w.result.index),
    description='Node:',
    ensure_option=True,
    disabled=False
)
kpis = list(r_util.get_kpi_names_from_pivot_table(w.result))
kpiForReport = widgets.Combobox(
    value=kpis[0],
    options = kpis,
    description='KPI:',
    ensure_option=True,
    disabled=False
)
surpassLimit = (0.0, 2.0, 0.1)
underpassLimit = (0.0, 2.0, 0.1)

s = interact(get_reports_per_kpi_per_node, pivot_table=fixed(w.result), nodeForReport=nodeForReport, kpiForReport=kpiForReport, surpassLimit=surpassLimit, underpassLimit=underpassLimit)
